Init

In [1]:
src_file = 'all.csv'

Import libs

In [2]:
import pandas as pd
import numpy as np

In [3]:
dataframe = pd.read_csv(src_file, quotechar="\"", engine='python', usecols=["TITLE", "CATEGORY"])
# dataframe = pd.read_csv(src_file, sep='\t', header=None, names=['ID', 'TITLE', 'URL', 'Source', 'CATEGORY', 'Key', 'Website', 'Timestamp'])

# all_categories = ["WORLD", WHITE HOUSE]
# print(dataframe.groupby("CATEGORY").size().reset_index(name='COUNT'))
category_count = dataframe.groupby("CATEGORY").size().reset_index(name='COUNT')
category_count = category_count[category_count["COUNT"] > 10]

dataframe = dataframe[dataframe["CATEGORY"].isin(category_count["CATEGORY"])]


# dataframe = dataframe[dataframe["CATEGORY"].isin(["U.S. NEWS", "NEWS", "WORLD"])]


print(dataframe.groupby("CATEGORY").size().reset_index(name='COUNT'))
print(dataframe.head())
dataframe.shape

# print(category_count = category_count[category_count["COUNT"] > 100])
# print(dataframe["CATEGORY"].unique())
# print(dataframe)
# print(dataframe.shape)

              CATEGORY  COUNT
0     #METOO RECKONING     12
1        2022 ELECTION     81
2        2024 ELECTION    165
3      ABORTION RIGHTS    123
4   AFTER GEORGE FLOYD     16
..                 ...    ...
90      WAR IN UKRAINE    509
91             WEATHER     55
92   WESTERN WILDFIRES     19
93         WHITE HOUSE    202
94               WORLD    813

[95 rows x 2 columns]
                                               TITLE         CATEGORY
0  A college professor called the police on two s...  CULTURE MATTERS
1  Oscars producer says police were prepared to a...        CELEBRITY
2  Jared Kushner interviewed by Jan. 6 committee ...     DONALD TRUMP
3  House passes bill to cap out-of-pocket insulin...         CONGRESS
4  Police shoot 'hero' after he disarms gunman, i...        U.S. NEWS


(11334, 2)

In [4]:
#check for missing data
if(any(dataframe.isnull().any())):
    print('Missing Data\n')
    print(dataframe.isnull().sum())
else:
    print('NO missing data')

NO missing data


In [5]:
# check for duplicate
if(any(dataframe.duplicated())==True):
    print('Duplicate rows found')
    print('Number of duplicate rows= ', dataframe[dataframe.duplicated()].shape[0])
    dataframe.drop_duplicates(inplace=True,keep='first')
    dataframe.reset_index(inplace=True,drop=True)
    print('Dropping duplicates\n')
    print(dataframe.shape)
else:
    print('NO duplicate data')

Duplicate rows found
Number of duplicate rows=  44
Dropping duplicates

(11290, 2)


In [6]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/misapisatto/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/misapisatto/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/misapisatto/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import string
from sklearn import set_config
set_config(transform_output="pandas")

wnl = WordNetLemmatizer()

# Function for cleaning and tokenize the headline
def tokenize(doc):
  document = doc.lower() # convert the content of the headline to lowercase
  document = re.sub(r'\d+', '', document) # remove all of the digits inside of the content (using regular expressions)
  document = document.translate(str.maketrans('', '', string.punctuation)) # remove the puntuations (, . ! # ...)
  document = document.strip() # remove the spaces at the start and end of the headline
  return [wnl.lemmatize(token) for token in word_tokenize(document) if token not in stopwords.words('english')]
  # tokenize the headlines
  # and then filter only the words that are not in the english stopwords (words that are commonly used and give no benifits to the classifier)
  # and finally lemmatize all of the tokens

# The preprocess pipeline
preprocessor = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)), # passing custom tokenizer method for the CountVectorizer to use
    ('tfidf', TfidfTransformer()),
])

tfidf_dataset = preprocessor.fit_transform(dataframe["TITLE"].values) # process the training dataset
# tfidf_test = preprocessor.transform(X_test.values) # process the testing dataset

/Users/misapisatto/Library/Python/3.9/lib/python/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [8]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
class_label = le.fit_transform(dataframe["CATEGORY"])

print(dataframe["CATEGORY"])
print(class_label)


0         CULTURE MATTERS
1               CELEBRITY
2            DONALD TRUMP
3                CONGRESS
4               U.S. NEWS
               ...       
11285          FIRST READ
11286     MORNING RUNDOWN
11287    ISRAEL-HAMAS WAR
11288           U.S. NEWS
11289      WAR IN UKRAINE
Name: CATEGORY, Length: 11290, dtype: object
[21 11 24 ... 45 89 90]


In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    tfidf_dataset.toarray(),
    class_label,
    test_size = 0.3 # the size of the testing dataset (in percentage between 0 and 1)
)

In [10]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

#Decision Tree
DTClass = DecisionTreeClassifier(criterion="gini", splitter="best", random_state=40)
DTClass.fit(X_train, y_train)
y_pred = DTClass.predict(X_test)

o = DTClass.predict(preprocessor.transform(["Slashing Central American aid could drive more migrants to the U.S."]).toarray())
print(o)
print(dataframe["CATEGORY"].values[np.where(class_label == o)[0]][0])

# print("accuracy score of Decision Tree:")
print(accuracy_score(y_test, y_pred))

[41]
IMMIGRATION
0.4301741954532034


In [11]:
o = DTClass.predict(preprocessor.transform(["Slashing Central American aid could drive more migrants to the U.S."]).toarray())
print(o)
print(dataframe["CATEGORY"].values[np.where(class_label == o)[0]][0])
print(accuracy_score(y_test, y_pred))


[41]
IMMIGRATION
0.4301741954532034


In [12]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

naive_bayes_model = MultinomialNB()

vectorizer = CountVectorizer()

X_train, X_test, y_train, y_test = train_test_split(dataframe['TITLE'], dataframe['CATEGORY'], test_size=0.1, random_state=42)
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

naive_bayes_model.fit(X_train_vectorized, y_train)

y_pred = naive_bayes_model.predict(X_test_vectorized)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)

print(accuracy)
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print("accuracy score of Naive Bayes:")

new_title_vectorized = vectorizer.transform(["At least five states are considering requiring full minimum wages for tip earners this year"])

predicted_category = naive_bayes_model.predict(new_title_vectorized)

print(f'Predicted Category: {predicted_category[0]}')
# print(accuracy)
# print(conf_matrix)
# print(classification_rep)

0.43932683790965454
accuracy score of Naive Bayes:
Predicted Category: U.S. NEWS


/Users/misapisatto/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/misapisatto/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/misapisatto/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

In [13]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

                         precision    recall  f1-score   support

          2022 ELECTION       1.00      0.12      0.22         8
          2024 ELECTION       0.62      0.33      0.43        15
        ABORTION RIGHTS       0.57      0.18      0.28        22
     AFTER GEORGE FLOYD       0.00      0.00      0.00         4
            ANIMAL NEWS       0.00      0.00      0.00         2
ARTIFICIAL INTELLIGENCE       0.00      0.00      0.00         4
          ASIAN AMERICA       1.00      0.18      0.30        17
                  AUTOS       0.00      0.00      0.00         1
          BUSINESS NEWS       0.50      0.10      0.16        21
           CAPITOL RIOT       0.00      0.00      0.00         3
              CELEBRITY       0.00      0.00      0.00         1
      CLIMATE IN CRISIS       1.00      0.08      0.15        12
    COLD CASE SPOTLIGHT       0.00      0.00      0.00         5
               CONGRESS       0.32      0.83      0.46        36
               CONSUMER 

/Users/misapisatto/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/misapisatto/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/misapisatto/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu